In [3]:
import requests
from geopy.geocoders import Nominatim
import time 

In [6]:
url = 'https://api.wheretheiss.at/v1/satellites/25544'
headers = {'Accepts': 'application/json'}

r = requests.get(url, headers=headers)
new = r.json()
new

{'name': 'iss',
 'id': 25544,
 'latitude': 13.241606453598,
 'longitude': -75.886306841649,
 'altitude': 414.31819818686,
 'velocity': 27600.735677592,
 'visibility': 'daylight',
 'footprint': 4478.4433001778,
 'timestamp': 1671903512,
 'daynum': 2459938.2350926,
 'solar_lat': -23.406616509741,
 'solar_lon': 275.28714632319,
 'units': 'kilometers'}

In [7]:
geolocator = Nominatim(user_agent="test")

while True:
    url = 'https://api.wheretheiss.at/v1/satellites/25544'
    headers = {'Accepts': 'application/json'}

    r = requests.get(url, headers=headers)
    new = r.json()
    new

    location = geolocator.reverse(f"{new['latitude']}, {new['longitude']}")

    if location is None:
        print("location unknown (must be a body of water), going to sleep...")
        time.sleep(30)
        continue 

    else:
        print(location.address)
        break 

location unknown (must be a body of water), going to sleep...


In [10]:
new['latitude']

-48.25010511899

In [11]:
new['longitude']

-15.146657999075

'-23.424155830858, 255.19063133778'